In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
df=pd.read_csv('GEOS566 HW1.csv')


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(df)

In [ ]:
daily=df.groupby('D')[['Dec','e2']].first()
daily=daily.interpolate().ffill().bfill()

df['Dec']=df['D'].map(daily['Dec'])
df['e2']=df['D'].map(daily['e2'])

print(df)

In [ ]:
#Fix units! Need radians for python trigonometric functions
df['Lat']=np.radians(df['Lat'])
df['Dec']=np.radians(df['Dec'])
df['H']=df['H']/100 #convert hours to 0-24 scale

print(df)

In [ ]:
#Defining constants

w=np.radians(15) # pi/12 rad per hour or 15 degrees per hour (angular velocity of Earth)
I0=1380 #W/m^2 (solar constant)

#Calculating cos(z)

df['cosz']=np.sin(df['Lat'])*np.sin(df['Dec']) + np.cos(df['Lat'])*np.cos(df['Dec'])*np.cos(w*(df['H']-12))


In [ ]:
#potential solar radiation

df['I']=I0*df['cosz']/df['e2']

df.loc[df['I']<0,'I']=0

In [ ]:
#plot incoming solar radiation for a week in winter and a week in spring

fig, ax = plt.subplots(figsize=(10,6))

winter=df[(df['D']>=15) & (df['D']<=21)].copy()
spring=df[(df['D']>=91) & (df['D']<=97)].copy()

winter['t']=24*(winter['D']-15)+winter['H']
spring['t']=24*(spring['D']-91)+spring['H']

ax.plot(winter['t'], winter['I'], label='Winter (January 15-21)', color='blue')
ax.plot(spring['t'], spring['I'], label='Spring (April 1-7)', color='red')
ax.set_xlabel('Hour since start of week')
ax.set_ylabel('Incoming Solar Radiation (W/m^2)')
ax.set_title('Incoming Solar Radiation for a Week in Winter versus Spring')
ax.legend()
plt.show()

In [ ]:
# What if true solar noon is at 12:24 pm? Then the hour difference from solar noon component needs to be H-12.4 instead of H-12. I don't think that 0.4 can have much of an impact inside a cosine function...

df['cosz']=np.sin(df['Lat'])*np.sin(df['Dec']) + np.cos(df['Lat'])*np.cos(df['Dec'])*np.cos(w*(df['H']-12.4))
df['I']=I0*df['cosz']/df['e2']
df.loc[df['I']<0,'I']=0

fig, ax = plt.subplots(figsize=(10,6))

winter=df[(df['D']>=15) & (df['D']<=21)].copy()
spring=df[(df['D']>=91) & (df['D']<=97)].copy()

winter['t']=24*(winter['D']-15)+winter['H']
spring['t']=24*(spring['D']-91)+spring['H']

ax.plot(winter['t'], winter['I'], label='Winter (January 15-21)', color='blue')
ax.plot(spring['t'], spring['I'], label='Spring (April 1-7)', color='red')
ax.set_xlabel('Hour since start of week')
ax.set_ylabel('Incoming Solar Radiation (W/m^2)')
ax.set_title('Incoming Solar Radiation for a Week in Winter versus Spring, adjusted solar noon')
ax.legend()
plt.show()

In [ ]:
#modifying for slope and aspect

def I_adj(Lat,Dec,H,slope,aspect,I0,e2):
    adjusted_Lat=np.arcsin(np.sin(slope)*np.cos(aspect)*np.cos(Lat) + np.cos(slope)*np.sin(Lat))
    adjusted_Long=np.arctan2((np.sin(aspect)*np.sin(slope)),((np.cos(slope)*np.cos(Lat))-(np.cos(aspect)*np.sin(slope)*np.sin(Lat))))
    adjusted_cosz=np.sin(adjusted_Lat)*np.sin(Dec) + np.cos(adjusted_Lat)*np.cos(Dec)*np.cos(w*(H-12)-adjusted_Long)
    adjusted_I=I0*adjusted_cosz/e2
    adjusted_I[adjusted_I<0]=0
    return adjusted_I



slope=np.radians(35) #35 degree slope
aspect=np.radians(0) #N-facing slope

df['I35N']=I_adj(df['Lat'],df['Dec'],df['H'],slope,aspect,I0,df['e2'])

aspect=np.radians(180) #S-facing slope

df['I35S']=I_adj(df['Lat'],df['Dec'],df['H'],slope,aspect,I0,df['e2'])

print(df.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
winter=df[(df['D']>=15) & (df['D']<=21)].copy()
spring=df[(df['D']>=91) & (df['D']<=97)].copy()
winter['t']=24*(winter['D']-15)+winter['H']
spring['t']=24*(spring['D']-91)+spring['H']
ax.plot(winter['t'], winter['I35N'], label='Winter 35° N-facing', color='blue')
ax.plot(winter['t'], winter['I35S'], label='Winter 35° S-facing', color='cyan')
ax.plot(spring['t'], spring['I35N'], label='Spring 35° N-facing', color='red')
ax.plot(spring['t'], spring['I35S'], label='Spring 35° S-facing', color='orange')
ax.set_xlabel('Hour since start of week')
ax.set_ylabel('Incoming Solar Radiation (W/m^2)')
ax.set_title('Incoming Solar Radiation for 35° Slopes Facing North and South')
ax.legend()
plt.show()


In [ ]:
slope=np.radians(15) #15 degree slope
aspect=np.radians(0) #N-facing slope

df['I15N']=I_adj(df['Lat'],df['Dec'],df['H'],slope,aspect,I0,df['e2'])

aspect=np.radians(180) #S-facing slope
df['I15S']=I_adj(df['Lat'],df['Dec'],df['H'],slope,aspect,I0,df['e2'])

fig, ax = plt.subplots(figsize=(10,6))
winter=df[(df['D']>=15) & (df['D']<=21)].copy()
spring=df[(df['D']>=91) & (df['D']<=97)].copy()
winter['t']=24*(winter['D']-15)+winter['H']
spring['t']=24*(spring['D']-91)+spring['H']
ax.plot(winter['t'], winter['I15N'], label='Winter 15° N-facing', color='blue')
ax.plot(winter['t'], winter['I15S'], label='Winter 15° S-facing', color='red')
ax.plot(spring['t'], spring['I15N'], label='Spring 15° N-facing', color='cyan')
ax.plot(spring['t'], spring['I15S'], label='Spring 15° S-facing', color='orange')
ax.set_xlabel('Hour since start of week')
ax.set_ylabel('Incoming Solar Radiation (W/m^2)')
ax.set_title('Incoming Solar Radiation for 15° Slopes Facing North and South')
ax.legend()
plt.show()